In [92]:
import settransducer
from presets import setmedium
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import numpy as np

In [93]:
df_pressure2D = pd.read_csv(r"C:\Users\geral\OneDrive\Documents\GitHub\TempCompliance4HIFU\src\matlab_data_1MHz_2cm_5cm.csv")  # sample_data
df_pressure2D = np.rot90(df_pressure2D)
z_axis = np.linspace(0,0.01,100)
r_axis = np.linspace(-0.02,0.02,101)

pressure2D = df_pressure2D
z_values = z_axis
r_values = r_axis

In [94]:
def calculateBioHeat(axial_location,radial_location,trans,medium,heat,pressure2D,z_values,r_values,iscomplete=0):
    """
    INPUT ARG
        trans == dictionary of transducer properties
            "freq" = [Hz]
            "radius" = [m]
            "focus" = [m]
            "initPressure" = [Pa]
        medium == calls medium properties
            "speed" = [m/s]
            "density" = [kg/m^3]
            "absCoeff" = [Np/(m*MHz^2)]
        heat == calls heating parameters
            "HeatTime"
            "CoolTime"
            "DutyCycle"
        

    OUTPUT ARG
        pressure_field == resulting rayleigh integral pressure field


    """
    

    # Edit and Transform Transducer Properties
    d = trans["focus"]
    k = 2 * np.pi * trans["freq"] / medium["speed"] # Wave Number
    angularF = trans["radius"]/trans["focus"] # Angular Frequency
    abs_Coeff = medium["absCoeff"] * (pow((trans["freq"]/(1e6)),2)) # POWER LAW
    # impedance = medium["speed"]/medium["density"]
    # gain = 2 * np.pi * trans["freq"] * ((trans["radius"])^2) / (2*trans["focus"]*medium["speed"])
    dutyCycleRatio = heat["DutyCycle"]/100

    # Set Axes
    axial_min = 0.001
    axial_max = 2*d
    radial_min = -trans["radius"]
    dz = (axial_max - axial_min) / (len(z_values)-1)
    print(dz)
    dr = -1 * radial_min / ((len(r_values)-1)/2)
    print(dr)
    temp_dist_times = [30]
    dt = 1
    time_axis = np.arange(0,(heat["HeatTime"]+heat["CoolTime"]+dt),dt)




    # Bioheat
    intensity2D = pow(pressure2D,2) * pow(trans["initPressure"],2) * dutyCycleRatio / (2 * medium["density"] * medium["speed"])
    Q = intensity2D * 2 * abs_Coeff / (medium["density"] * medium["specHeatCap"])

    # Temperature Matrix
    num_dist = 0
    numR,numZ = np.shape(pressure2D)
    temp_dist = np.zeros((numR,numZ))

    r_center_idx = round(((0-radial_min)/dr)+1)
    radial_temp_idx = round(((radial_location-radial_min)/dr)+1)
    axial_temp_idx = round(((axial_location-axial_min)/dz)+1)
    temp_vec = []
    temp_vec.append(temp_dist[axial_temp_idx,radial_temp_idx])

    heat_dist_indices = []
    cool_dist_indices = []
    for ii in range(1,len(temp_dist_times)):
        time_value = temp_dist_times[ii]
        if time_value >= 0 and time_value <= heat["HeatTime"]:
            heat_dist_indices = heat_dist_indices.append(round(time_value/dt))
        elif time_value > heat["HeatTime"]and time_value <= (heat["HeatTime"] + heat["CoolTime"]):
            cool_dist_indices = cool_dist_indices.append(round((time_value - heat["HeatTime"])/dt))
    

    # FTCS Scheme ADD FUNCTION FOR HEAT
    term1 = ((dt * medium["thermDiff"]) / pow(dr,2))
    for time_step in np.arange(dt,heat["HeatTime"],dt):
        curr_temp_dist = temp_dist
        for ii in range(1,numR-1):
            for jj in range(1, numZ-1):
                currTemp = curr_temp_dist[ii,jj]

                term2z = (curr_temp_dist[ii+1,jj] - (2 * curr_temp_dist[ii,jj]) + curr_temp_dist[ii-1,jj])
                term2r = (curr_temp_dist[ii,jj+1] - (2 * curr_temp_dist[ii,jj]) + curr_temp_dist[ii,jj-1])
                term3r = (1/r_values[jj]) * (curr_temp_dist[ii,jj+1] - curr_temp_dist[ii,jj-1]) * ((dt * medium["thermDiff"]) / (2*dr))
                
                z_component = term1 * term2z
                r_component = term1 * term2r * term3r

                if jj == r_center_idx:
                    r_component = 4*term1*(curr_temp_dist[ii,jj+1] - curr_temp_dist[ii,jj])

                Q_US = dt * Q[ii,jj]
                temp_dist[ii,jj] = currTemp + z_component + r_component + Q_US
        temp_vec.append(temp_dist[axial_temp_idx,radial_temp_idx])
            

    # FTCS Scheme ADD FUNCTION FOR COOL
    for time_step in np.arange(dt,heat["CoolTime"],dt):
        curr_temp_dist = temp_dist
        for ii in range(1,numR-2):
            for jj in range(1, numZ-2):
                currTemp = curr_temp_dist[ii,jj]

                z_component = ((dt * medium["thermDiff"]) / pow(dr,2)) \
                * (curr_temp_dist[ii+1,jj] - (2 * curr_temp_dist[ii,jj]) + curr_temp_dist[ii-1,jj]) 

                r_component = ((dt * medium["thermDiff"]) / pow(dr,2)) \
                * (curr_temp_dist[ii,jj+1] - (2 * curr_temp_dist[ii,jj]) + curr_temp_dist[ii,jj-1]) \
                + (1/r_values[jj]) * (curr_temp_dist[ii,jj+1] - curr_temp_dist[ii,jj-1]) \
                * (dt * medium["thermDiff"] / (2*dr))

                if jj == r_center_idx:
                    r_component = 4*((dt * medium["thermDiff"]) / pow(dr,2)) \
                    * (curr_temp_dist[ii,jj+1] - curr_temp_dist[ii,jj])

                temp_dist[ii,jj] = currTemp + z_component + r_component
        temp_vec.append(temp_dist[axial_temp_idx,radial_temp_idx])




    print("Bioheat Calculation Complete")
    iscomplete = 1
    return time_axis, temp_vec, iscomplete, Q


In [95]:
import calculateRayleighIntegral

trans = settransducer.setTransducer(1e6,0.02,0.05,1) # freq, radius, focus, init pressure
medium = setmedium.setMedium('Water', dict())
field = dict(numAxialStep = 100, numRadialStep = 100)

# Rayleigh Integral
df_pressure2D, z_axis, r_axis, iscomplete = calculateRayleighIntegral.generateField(trans,medium,field)

1.0 %
2.0 %
3.0 %
4.0 %
5.0 %
6.0 %
7.0 %
8.0 %
9.0 %
10.0 %
11.0 %
12.0 %
13.0 %
14.0 %
15.0 %
16.0 %
17.0 %
18.0 %
19.0 %
20.0 %
21.0 %
22.0 %
23.0 %
24.0 %
25.0 %
26.0 %
27.0 %
28.0 %
29.0 %
30.0 %
31.0 %
32.0 %
33.0 %
34.0 %
35.0 %
36.0 %
37.0 %
38.0 %
39.0 %
40.0 %
41.0 %
42.0 %
43.0 %
44.0 %
45.0 %
46.0 %
47.0 %
48.0 %
49.0 %
50.0 %
50.0 %
51.0 %
52.0 %
53.0 %
54.0 %
55.0 %
56.0 %
57.0 %
58.0 %
59.0 %
60.0 %
61.0 %
62.0 %
63.0 %
64.0 %
65.0 %
66.0 %
67.0 %
68.0 %
69.0 %
70.0 %
71.0 %
72.0 %
73.0 %
74.0 %
75.0 %
76.0 %
77.0 %
78.0 %
79.0 %
80.0 %
81.0 %
82.0 %
83.0 %
84.0 %
85.0 %
86.0 %
87.0 %
88.0 %
89.0 %
90.0 %
91.0 %
92.0 %
93.0 %
94.0 %
95.0 %
96.0 %
97.0 %
98.0 %
99.0 %
Rayleigh Integral Complete


In [103]:
print(df_pressure2D)
len(df_pressure2D)


[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.65829078 0.46899967 ... 0.48885699 0.4896785  0.49023066]
 [0.         0.76288511 0.60846114 ... 0.48266104 0.48416033 0.48764391]
 ...
 [0.         0.76288511 0.60846114 ... 0.48266104 0.48416033 0.48764391]
 [0.         0.65829078 0.46899967 ... 0.48885699 0.4896785  0.49023066]
 [0.         0.         0.         ... 0.         0.         0.        ]]


102

In [97]:
fig = px.imshow(
    df_pressure2D,
    x = z_axis,
    y = r_axis,
    aspect='auto',
    color_continuous_scale='jet', 
)
fig.show()

ValueError: The length of the y vector must match the length of the first dimension of the img matrix.

In [96]:
heat = dict(HeatTime = float(30), CoolTime= float(30), DutyCycle = int(100))
axial_location = 0.07 # [m]
radial_location = 0

time_axis, temp_vec, iscomplete, Q = calculateBioHeat(axial_location,radial_location,trans,medium,heat,df_pressure2D,z_axis,r_axis,iscomplete=0)


0.00099
0.0004


C:\Users\geral\AppData\Local\Temp\ipykernel_18596\4038337937.py:94: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)

C:\Users\geral\AppData\Local\Temp\ipykernel_18596\4038337937.py:88: RuntimeWarning:

overflow encountered in multiply

C:\Users\geral\AppData\Local\Temp\ipykernel_18596\4038337937.py:85: RuntimeWarning:

overflow encountered in multiply

C:\Users\geral\AppData\Local\Temp\ipykernel_18596\4038337937.py:83: RuntimeWarning:

invalid value encountered in scalar subtract

C:\Users\geral\AppData\Local\Temp\ipykernel_18596\4038337937.py:84: RuntimeWarning:

invalid value encountered in scalar add

C:\Users\geral\AppData\Local\Temp\ipykernel_18596\4038337937.py:85: RuntimeWarning:

invalid value encountered in scalar subtract

C:\Users\geral\AppData\Local\Temp\ipykernel_18596\4038337937.py:84: RuntimeWarning:

KeyboardInterrupt: 

In [ ]:
print(temp_vec)

[0.0, 9.09566773993451e-13, -1.8424063311917534e-12, 1.404165544210902e-10, -3.5743023822290034e-09, 7.000025785210759e-08, -1.094795227557579e-06, 1.408785090599183e-05, -0.00014960083757051875, 0.0012558241466676613, -0.006575201703833003, -0.029123833850697237, 1.503357622172835, -64.5560103277043, -22726.150697048564, 2385857963960.671, 6.1688427680253004e+44, 6.9862484257366574e+174, inf, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]


In [ ]:
fig2D = px.line(
        temp_vec, 
        # x="Time [s]", y="Temperature [Degrees Celcius]",
    )
fig2D.show()